<a href="https://colab.research.google.com/github/Zamoca42/TIL/blob/main/Python/Dynamic_Crawling_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 동적 페이지 크롤링

## 동적 크롤링 챕터에서 배울 내용

1. 브라우저, 셀레니움 소개
2. 브라우저 원리
3. 네이버 쇼핑 검색 및 구매
4. 이미지 업로드
5. 자주 나는 오류 FAQs 및 한계

## 브라우저가 하는 일

1. 렌더링
  - html문서를 화면에 나타내는 것
2. 편의기능
  - 북마크 
  - 비밀번호 채워주기
  - 보안관리
  - 동기화
  - 개인화

### 예시 : 블로그 글 보기/쓰기

1. 네이버 접속(렌더링)
2. 블로그 버튼 클릭(인터렉션)
3. 블로그 페이지 접속(렌더링)
4. 블로그 검색(인터렉션)
5. 블로그 글 검색(렌더링)

</br>

1. 블로그 글쓰기 클릭
2. 글 입력(제목, 본문, 태그, 사진 등)
3. 작성완료 버튼

## 셀레니움

### 셀레니움이란?

- 브라우저 테스팅 툴
- 브라우저 원격 조종 툴

- 예시 : 페이스북 개발, 무한 스크롤이 잘 작동하는지 테스트?

- 사람이 직접 테스트
  - QA = 인력이 많이듬, 반복적 작업에 비효율적
- 컴퓨터 자동화 테스트 
  - 셀레니움 = 인력최소화, 반복적 작업에 매우 효율적

### 셀레니움 원리
- 디버깅 모드의 브라우저와 TCP 통신, 마치 리모컨

## 브라우저 원리

### 정적웹사이트 vs 동적 웹사이트

- 정적 웹사이트 
  - 움직임이 없는, 고정된 웹사이트

- 동적 웹사이트
  - 움직임이 많음, 다이나믹 웹사이트

### 정적 웹사이트

1. 데이터가 한번에 담겨져온다
2. 중복된 데이터가 잦다
3. 드물게 통신한다
4. 역사가 오래되어 안정적

### 동적 웹사이트

1. 사이트가 깜빡이지 않는다
2. 화면이 한번에 다 로딩되지 않는다
  - DOM생성
3. javascript가 필수다

- 서버 <-> 클라이언트 흐름 및 특징  
  1. 데이터가 따로 따로 여러군데서 온다
  2. 서버와 자주 통신한다
  3. 디자인이 화려하다
  4. 기능이 많고 사용성이 편리하다

## 네이버 쇼핑 검색하기

### 페이지 이동

In [ ]:
#이 부분은 처음 한번만 실행하면 됌.
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
# -*- coding: UTF-8 -*-
import time
from selenium import webdriver

#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
 
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
chrome = webdriver.Chrome('chromedriver', options=options)

In [ ]:
#해당 url로 이동
url = "https://www.naver.com/" 

chrome.get(url)

print(chrome.title)

NAVER


In [ ]:
url2 = "https://shopping.naver.com/" 
url = "https://www.naver.com"

chrome.get(url)
chrome.get(url2)
chrome.back() # 페이지 뒤로 이동
chrome.forward() # 페이지 앞으로 이동
time.sleep(3)
chrome.close()

### 페이지 로딩

In [ ]:
# selenium
# ...
    def execute(self, driver_command: str, params: dict = None) -> dict:
        """
        Sends a command to be executed by a command.CommandExecutor.

        :Args:
         - driver_command: The name of the command to execute as a string.
         - params: A dictionary of named parameters to send with the command.

        :Returns:
          The command's JSON response loaded into a dictionary object.
        """
        params = self._wrap_value(params)

        if self.session_id:
            if not params:
                params = {"sessionId": self.session_id}
            elif "sessionId" not in params:
                params["sessionId"] = self.session_id

        response = self.command_executor.execute(driver_command, params)
        if response:
            self.error_handler.check_response(response)
            response["value"] = self._unwrap_value(response.get("value", None))
            return response
        # If the server doesn't send a response, assume the command was
        # a success
        return {"success": 0, "value": None, "sessionId": self.session_id}

    def get(self, url: str) -> None:
        """
        Loads a web page in the current browser session.
        """
        self.execute(Command.GET, {"url": url})


- selenium의 get함수 일부
  - selenium의 브라우저에 HTTP 요청을 보낸다

### Explicit Waits vs Implicit Waits vs time.sleep

- Implicit Waits
  - 일정 시간만큼을 기다림 (주로 페이지 로딩 시간을 기다림)
  - 페이지가 2초만에 로딩된다면, 10초를 기다리지않고 바로 다음 코드로 넘어감

- Explicit Waits
  - 조건을 추가하고 조건에 맞춰서 기다림
  - 주어진 시간을 넘기면 에러를 띄운다
    - NoSuchElementException
    - ElementNotVisibleException

- time.sleep(10)
  - 무조건 10초를 기다림


In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

url = "https://m.shopping.naver.com/home" 

chrome.get(url)
# 3초 기다림
time.sleep(3)
# 페이지가 로딩될 때까지 3초간 기다림
chrome.implicity_wait(3)

chrome.close()

- WebDriverWait를 이용한 Explicit Waits
  - `WebDriverWait(driver, 10)`
    - WebDriverWait을 통해 driver를 "최대" 10초동안 기다림
  - `.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[name=query]"))`
    - CSS_SELECTOR가 "myDynamicElement"인 엘리먼트가 나올 때까지
  

In [ ]:
# Explicit waits
WebDriverWait(chrome, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[name=query]")))

- CSS_SELECTOR가 `"input[name=query]"`인 element가 나올 때까지 최대 10초를 기다린다.

### Element 찾기


In [ ]:
from selenium.webdriver.support import expected_conditions as EC

- expected_conditions의 속성값
  - 속성이름과 값이 튜플 형태
  - `alert_is_present()`
    - alert의 유무를 체크
  - `all_of_expected_conditions(*expected_conditions)`
    - 파라미터로 ExpectedConditions 객체를 가변인자로 넣으면 여러 상태를 동시에 체크
    - AND 연산자와 비슷
  - `any_of_expected_conditions(*expected_conditions)`
    - 파라미터로 ExpectedConditions 객체를 가변인자로 넣으면 여러 상태 중 하나만 True일때 True를 반환
    - OR 연산자와 비슷
  - `element_attribute_to_include(locator, attribute_)`
    - 해당 attribute_의 속성이 locator에 포함되어 있는지 체크
  - `title_is(title)`
    - 페이지 제목을 확인하기 위한 함수
    - 현재 페이지 제목이 같다면True를, 아니라면 False를 반환
  - `title_contains(title)`
    - 페이지 제목을 확인
    - 대/소문자를 구분
    - 같으면 true, 다르면 false

  - `presence_of_element_located(locator)`
    - locator로 들어간 element가 DOM에 있는지 확인
    - element가 존재하면 true, 없다면 false
  
  - `presence_of_all_elements_located(locator)`
    - locator로 들어간 element가 하나 이상 존재하는지 확인
    - 찾는 element들을 리스트로 반환

  - `visibility_of(locator)`
    - locator로 들어간 element가 보이는지 확인
    - hidden등의 속성 값으로 보이지 않는 element들을 체크하는데 사용
    - 해당 element가 보인다면 True, 그렇지 않다면 False를 반환
  - `visibility_of_element_located(locator)`
    - locator로 들어간 element가 보이는지 체크
    - DOM에도 있는지 체크
  - `invisibility_of_element_located(locator)`
    - locator로 들어간 element가 보이지 않다면 true, 보인다면 false

- find element
  -  셀레늄에서 다양한 방식으로 웹페이지의 요소(element)를 찾을 수 있는 방법중 하나

| 단일 element	| 복수 elements	| 설명 |
| :--- | :--- | :--- |
| find_element_by_id |   | 태그의 id값으로 추출|
|find_element_by_name|find_elements_by_name|태그의 name값으로 추출|
|find_element_by_xpath	| find_elements_by_xpath	|태그의 경로로 추출|
|find_element_by_link_text |	find_elements_by_link_text|	a태그 텍스트값으로 추출|
|find_element_by_partial_link_text	|find_elements_by_partial_link_text	|a태그의 자식 텍스트 값을 추출|
|find_element_by_tag_name|	find_elements_by_tag_name	|태그 이름으로 추출|
|find_element_by_class_name	|find_elements_by_class_name|	태그의 클래스명으로 추출|
|find_element_by_css_selector	|find_elements_by_css_selector	|css선택자로 추출

- By
  - find element와 같이 웹페이지의 요소를 찾는 방법
  - 유지 관리하기 쉬움

|element(단일) , elements(복수)	|설명|
|:---|:---|
|By.ID	| 태그의 id값으로 추출 |
|By.NAME	| 태그의 name값으로 추출 |
|By.XPATH	| 태그의 경로로 추출 |
|By.LINK_TEXT	| 링크 텍스트값으로 추출 |
|By.PARTIAL_LINK_TEXT	| 링크 텍스트의 자식 텍스트 값을 추출 |
|By.TAG_NAME	| 태그 이름으로 추출 |
|By.CLASS_NAME	| 태그의 클래스명으로 추출 |
|By.CSS_SELECTOR	| css선택자로 추출 |

### 최종 코드

1. 네이버 쇼핑 페이지에 접속
2. "아이폰 케이스" 검색어 입력
3. 브라우저로 검색 결과를 보여주고 종료

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
 
# 모바일 페이지로 변경
mobile_emulation = { "deviceName": "iPhone X" }
options = webdriver.ChromeOptions()
# options.add_argument('--headless')        # Head-less 설정
# options.add_argument("window-size-279,1000")
options.add_argument("no-sandbox")
options.add_experimental_option("mobileEmulation", mobile_emulation)
# options.add_argument('disable-dev-shm-usage')
chrome = webdriver.Chrome('./chromedriver', options=options)

#해당 url로 이동
url = "https://m.shopping.naver.com/home" 

chrome.get(url)

# Explicit waits
wait = WebDriverWait(chrome, 10)

def find(wait, css_selector):
    return wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))

search = find(wait, "input[name=query]")

# 검색어 입력 후 Enter
search.send_keys("아이폰 케이스\n")

time.sleep(5)

# 버튼 찾아서 클릭
# button = find(wait, "button._gnb_button_search_qteu8.N\=a\:scb\.search")
# button.click()
# time.sleep(3)

chrome.close()

- 검색할 때 버튼의 element를 찾아서 `.click()`을 해줄 수 도 있지만  
`send_keys("아이폰 케이스\n")` 와 같이 `\n`으로 바로 엔터키를 입력하는 방법으로도 가능 

- 출처
  - https://unh6uoj.com/crawling/selenium-%EC%83%81%ED%83%9C-%ED%99%95%EC%9D%B8-ExpectedConditions-Class/
  - https://pythonblog.co.kr/coding/22/
  - https://wkdtjsgur100.github.io/selenium-xpath/
  - https://private.tistory.com/127